<div style='float: right'><img src='pic/sudoku.png'/></div>
## <div id='sudoku' />数独

In [ ]:
import numpy as np
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
..6.....1
.7..6..5.
8..1.32..
..5.4.8..
.4.7.2.9.
..8.1.7..
..12.5..3
.6..7..8.
2.....4..""".split()

### 問題
* マスに 1～9 の数字を入れます
* 縦、横または3×3に同じ数字は入りません

### 定式化
\begin{array}{cl}
            変数 & v_{ijk} \in \{0, 1\} ~ \forall i, j, k ~ ~ ~ マスi,jが数字k+1か (1) \\
\mbox{subject to} & \sum_k{v_{ijk}} = 1 ~ \forall i, j ~数字は1つ (2) \\
                 & \sum_k{v_{ikj}} = 1 ~ \forall i, j ~ 縦に同じ数字はない (3) \\
                 & \sum_k{v_{kij}} = 1 ~ \forall i, j ~ 横に同じ数字はない (4) \\
                 & 3\times3のマスについても同様 (5) \\
                 & 数字指定 (6) \\
\end{array}

In [ ]:
m = LpProblem()
v = np.array(addbinvars(9, 9, 9)) # (1)
for i in range(9):
    for j in range(9):
        m += lpSum(v[i,:,j]) == 1 # num (2)
        m += lpSum(v[i,j,:]) == 1 # col (3)
        m += lpSum(v[:,i,j]) == 1 # row (4)
        k, l = i//3*3, i%3*3
        m += lpSum(v[k:k+3,j,l:l+3]) == 1 # 3x3 (5)
        if data[i][j].isdigit():
            m += v[i,int(data[i][j])-1,j] == 1 # fix (6)
%time m.solve()
(np.arange(9) @ np.vectorize(value)(v).astype(int)+1).tolist()